This first cell will consist of a search for drugs that are geared for Alzheimers Disease and its late-onset.

In [21]:
#!/usr/bin/env python3

# Import relevant libraries to make HTTP requests and parse JSON response
import requests
import json
import pandas as pd
#def Disease_Ids():
# Set gene_id variable for AR (androgen receptor)
gene_id = "ENSG00000169083"
diseases_dict = {'Alzheimers': ["MONDO_0004975", "EFO_1001870"],
                 'Parkinsons': ["MONDO_0005180", "MONDO_0021095","Orphanet_411602"]}

## Displays the items inside the list object stored as the key-value pair for each disease
print(diseases_dict['Alzheimers'], diseases_dict['Parkinsons']) #displays two seperate lists

## Concatenates the disease ID's to create one list to use in the query string
efoIds = diseases_dict['Alzheimers']+ diseases_dict['Parkinsons']
print(efoIds) #display list as one array of efoIds
QueryString(efoIds)
# Disease_Ids()

['MONDO_0004975', 'EFO_1001870'] ['MONDO_0005180', 'MONDO_0021095', 'Orphanet_411602']
['MONDO_0004975', 'EFO_1001870', 'MONDO_0005180', 'MONDO_0021095', 'Orphanet_411602']


In [16]:
#def QueryString(efoIds):
#Build query string to get general information about AR and genetic constraint and tractability assessments 
query = '''
    query getInfoOnAlzheimers($efoIds: [String!]!) {
      diseases(efoIds: $efoIds) {
        id
        name
        dbXRefs
        knownDrugs {
          uniqueDrugs
          uniqueTargets
          rows {
            drug {
              id
              name
              tradeNames
            }
          }
        }
      }
    }
'''

# Set base URL of GraphQL API endpoint
base_url = "https://api.platform.opentargets.org/api/v4/graphql"

# Perform POST request and check status code of response
r = requests.post(base_url, json = {"query": query, "variables": {"efoIds": efoIds}})

assert r.status_code == 200, f'Expected status code is 200, got {r.status_code} instead'


In [17]:
# Extract data as json:
data = r.json()

# This variable will collect disease and drug data in a nicer format suitable to convert to pandas:
parsed_disease_data = []

In [22]:
## Creating a file to store ChEMBLIds into
f = open('chembls.txt','w')

# Looping over all diseases in the returned object:
for disease in data['data']['diseases']:
    disease_data = {
        'diseaseId': disease['id'],
        'diseaseName': disease['name'],
        'diseaseDbXRefs': disease['dbXRefs'],
    }
    #print(disease_data)
    ChEMBLIds = []
    # Further steps depends on the existence of drug object:
    if disease['knownDrugs']:
        disease_data.update({
            'uniqueDrugs': disease['knownDrugs']['uniqueDrugs'],
            'uniqueTargets': disease['knownDrugs']['uniqueTargets']
        })

        # Iterate over all drugs:
        drugs = []
        for drug in disease['knownDrugs']['rows']:
            drugs.append({
                'drugId': drug['drug'].get('id', None),
                'drugName': drug['drug'].get('name', None),
                'tradenames': drug['drug'].get('tradeNames', None)
            })
        
        ### Creates a local dataframe containing only drug information
        df = pd.DataFrame(drugs)
        ChEMBLIds = df['drugId'].tolist() # stores ChEMBLIds to a list
        
        ### Writes the list of drug information to a file
        file_data = "\n>" + disease_data['diseaseName'] + ":" + disease_data['diseaseId'] + "\n" #%s\n" % ChEMBLIds
        #print(file_data)
        f.write(file_data)
        for item in ChEMBLIds:
            f.write("\n" + item)
            #print(item)
        
        # Combining disease and drug data:
        disease_data.update({'drugs': drugs})

    # Collecting data:
    parsed_disease_data.append(disease_data)
f.close()

with open ('chembls.txt', 'r') as fp:
    #fp.readlines()
    print(fp.read())


>Alzheimer disease:MONDO_0004975

CHEMBL807
CHEMBL807
CHEMBL1699
CHEMBL3707183
CHEMBL807
CHEMBL1294
CHEMBL1678
CHEMBL502
CHEMBL807
CHEMBL1699
CHEMBL1555
CHEMBL636
CHEMBL723
CHEMBL502
CHEMBL807
CHEMBL1699
CHEMBL3707183
CHEMBL636
CHEMBL112
CHEMBL807
CHEMBL3
CHEMBL807
CHEMBL1699
CHEMBL1294
CHEMBL1699
>Parkinson disease:MONDO_0005180

CHEMBL900
CHEMBL45
CHEMBL517712
CHEMBL2103827
CHEMBL1569
CHEMBL589
CHEMBL953
CHEMBL887
CHEMBL1200503
CHEMBL1201023
CHEMBL1616
CHEMBL526
CHEMBL660
CHEMBL1569
CHEMBL1201023
CHEMBL1569
CHEMBL45
CHEMBL660
CHEMBL1569
CHEMBL1616
CHEMBL301265
CHEMBL526
CHEMBL1569
CHEMBL1200748
CHEMBL660
>parkinsonian disorder:MONDO_0021095

CHEMBL900
CHEMBL45
CHEMBL517712
CHEMBL2103827
CHEMBL1569
CHEMBL589
CHEMBL953
CHEMBL887
CHEMBL1200503
CHEMBL1201023
CHEMBL1616
CHEMBL526
CHEMBL660
CHEMBL1569
CHEMBL1201023
CHEMBL1569
CHEMBL45
CHEMBL660
CHEMBL1569
CHEMBL1616
CHEMBL301265
CHEMBL526
CHEMBL1569
CHEMBL1200748
CHEMBL660


In [23]:
df = (
    # Converting to pandas dataframe:
    pd.DataFrame(parsed_disease_data)
    # Exploding drug column to multiple rows:
    .explode('drugs')
    # Resetting index:
    .reset_index(drop=True)
)
df
# (
#     df
#     # Dropping rows with missing drug identifier:
#     .loc[lambda df: df.drugId.notna()]
#     # Select column:
#     .drugId
#     # Remove duplicates:
#     .drop_duplicates()
#     # Save list without index and header:
#     .to_csv('drugids.txt', index=False, header=None)
# )

,diseaseId,diseaseName,diseaseDbXRefs,uniqueDrugs,uniqueTargets,drugs
0,MONDO_0004975,Alzheimer disease,"[MESH:D000544, ICD10CM:G30, HP:0002511, DOID:1...",185.0,186.0,"{'drugId': 'CHEMBL807', 'drugName': 'MEMANTINE..."
1,MONDO_0004975,Alzheimer disease,"[MESH:D000544, ICD10CM:G30, HP:0002511, DOID:1...",185.0,186.0,"{'drugId': 'CHEMBL807', 'drugName': 'MEMANTINE..."
2,MONDO_0004975,Alzheimer disease,"[MESH:D000544, ICD10CM:G30, HP:0002511, DOID:1...",185.0,186.0,"{'drugId': 'CHEMBL1699', 'drugName': 'MEMANTIN..."
3,MONDO_0004975,Alzheimer disease,"[MESH:D000544, ICD10CM:G30, HP:0002511, DOID:1...",185.0,186.0,"{'drugId': 'CHEMBL3707183', 'drugName': 'QUINI..."
4,MONDO_0004975,Alzheimer disease,"[MESH:D000544, ICD10CM:G30, HP:0002511, DOID:1...",185.0,186.0,"{'drugId': 'CHEMBL807', 'drugName': 'MEMANTINE..."
...,...,...,...,...,...,...
149,MONDO_0021095,parkinsonian disorder,[MESH:D020734],158.0,158.0,"{'drugId': 'CHEMBL526', 'drugName': 'PROPOFOL'..."
150,MONDO_0021095,parkinsonian disorder,[MESH:D020734],158.0,158.0,"{'drugId': 'CHEMBL1569', 'drugName': 'AMANTADI..."
151,MONDO_0021095,parkinsonian disorder,[MESH:D020734],158.0,158.0,"{'drugId': 'CHEMBL1200748', 'drugName': 'CARBI..."
152,MONDO_0021095,parkinsonian disorder,[MESH:D020734],158.0,158.0,"{'drugId': 'CHEMBL660', 'drugName': 'AMANTADIN..."


In [53]:
prolog    = "https://pubchem.ncbi.nlm.nih.gov/rest/pug"
pr_input  = "compound/xref/RegistryID"#"compound/cid/129825914"
pr_oper   = "cids"
pr_output = "txt"

lines = []
# with open('chembls.txt', 'r') as file:
#     for line in file: 
#         line = line.strip() #or some other preprocessing
#         lines.append(line) #storing everything in memory!
        
disease_ids = {}
name = 0
ids = []
# with open ('chembls.txt', 'r') as fp:
#     for line in fp:
#         line = line.strip() #or some other preprocessing
#         lines.append(line) #storing everything in memory!
#         for line in lines:
            
#             if line.startswith('>'):
#                 name = line.strip()
#             elif line.startswith('CHEMBL'):
#                 id = line.strip()
#                 ids.append(id)
#         disease_cids.update({name:ids})
#     ids = [line.strip() for line in fp if line.startswith('CHEMBL')]
#     for id in ids:
#         url = prolog + '/' + pr_input + '/' + id + '/' + pr_oper + '/' + pr_output
#         res = requests.get(url)
#         print(id, ':' ,res.text)
    # print(len(ids))

with open ('chembls.txt', 'r') as fp:
    for line in fp:
        line = line.strip() #or some other preprocessing
        lines.append(line) #storing everything in memory!
    name = [line.strip() for line in fp if line.startswith('>')]
    print(name)
    ids = [line.strip() for line in fp if line.startswith('CHEMBL')]
    for id in ids:
        url = prolog + '/' + pr_input + '/' + id + '/' + pr_oper + '/' + pr_output
        res = requests.get(url)
        print(id, ':' ,res.text)
    disease_ids = {name:ids for x in name}
    
fp.close()
print(disease_ids)

# struct = {'xref':ids}

# url = prolog + '/' + pr_input + '/' + pr_oper + '/' + pr_output
# res = requests.post(url, params = struct)
# print(res.text)



[]
{}


In [164]:
prolog    = "https://pubchem.ncbi.nlm.nih.gov/rest/pug"
pr_input  = "compound/xref/RegistryID/"#"compound/cid/129825914"
pr_oper   = "cids"
pr_output = "txt"

with open ('chembls.txt', 'r') as fp:
    for line in fp:
        if line.startswith('CHEMBL'):# or line.startswith(''):
            print('yes')
            print(str(line))
            url = prolog + '/' + pr_input + "%s" %line + '/' + pr_oper + '/' + pr_output
            res = requests.get(url)
            open(res.text, 'w')
        #else:
            #pass
        
url = prolog + '/' + pr_input + "CHEMBL807" + '/' + pr_oper + '/' + pr_output
res = requests.get(url)
print(res.text)

yes
CHEMBL807

<!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//EN">
<html><head>
<title>400 Bad Request</title>
</head><body>
<h1>Bad Request</h1>
<p>Your browser sent a request that this server could not understand.<br />
</p>
</body></html>

yes
CHEMBL807

<!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//EN">
<html><head>
<title>400 Bad Request</title>
</head><body>
<h1>Bad Request</h1>
<p>Your browser sent a request that this server could not understand.<br />
</p>
</body></html>

yes
CHEMBL1699

<!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//EN">
<html><head>
<title>400 Bad Request</title>
</head><body>
<h1>Bad Request</h1>
<p>Your browser sent a request that this server could not understand.<br />
</p>
</body></html>

yes
CHEMBL3707183

<!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//EN">
<html><head>
<title>400 Bad Request</title>
</head><body>
<h1>Bad Request</h1>
<p>Your browser sent a request that this server could not understand.<br />
</p>
</body></html>

yes
CHEMBL807

<!DOCTYPE HT

<!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//EN">
<html><head>
<title>400 Bad Request</title>
</head><body>
<h1>Bad Request</h1>
<p>Your browser sent a request that this server could not understand.<br />
</p>
</body></html>

yes
CHEMBL1616

<!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//EN">
<html><head>
<title>400 Bad Request</title>
</head><body>
<h1>Bad Request</h1>
<p>Your browser sent a request that this server could not understand.<br />
</p>
</body></html>

yes
CHEMBL526

<!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//EN">
<html><head>
<title>400 Bad Request</title>
</head><body>
<h1>Bad Request</h1>
<p>Your browser sent a request that this server could not understand.<br />
</p>
</body></html>

yes
CHEMBL660

<!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//EN">
<html><head>
<title>400 Bad Request</title>
</head><body>
<h1>Bad Request</h1>
<p>Your browser sent a request that this server could not understand.<br />
</p>
</body></html>

yes
CHEMBL1569

<!DOCTYPE HTML PUBLIC "-//IETF

<!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//EN">
<html><head>
<title>400 Bad Request</title>
</head><body>
<h1>Bad Request</h1>
<p>Your browser sent a request that this server could not understand.<br />
</p>
</body></html>

yes
CHEMBL1616

<!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//EN">
<html><head>
<title>400 Bad Request</title>
</head><body>
<h1>Bad Request</h1>
<p>Your browser sent a request that this server could not understand.<br />
</p>
</body></html>

yes
CHEMBL301265

<!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//EN">
<html><head>
<title>400 Bad Request</title>
</head><body>
<h1>Bad Request</h1>
<p>Your browser sent a request that this server could not understand.<br />
</p>
</body></html>

yes
CHEMBL526

<!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//EN">
<html><head>
<title>400 Bad Request</title>
</head><body>
<h1>Bad Request</h1>
<p>Your browser sent a request that this server could not understand.<br />
</p>
</body></html>

yes
CHEMBL1569

<!DOCTYPE HTML PUBLIC "-//I

In [163]:

prolog    = "https://pubchem.ncbi.nlm.nih.gov/rest/pug"
pr_input  = "compound/name/"#"compound/cid/129825914"
pr_oper   = "sids"
pr_output = "txt"
drugs = {'quinidine', 'memantine', 'donepezil'}
for drug in drugs:
    url       = prolog + '/' + pr_input + drug + '/' + pr_oper + '/' + pr_output
    res = requests.get(url)
    
    print('For Drug: ', drug)
    #print(res.text.split())
    sids = res.text.split()
    
    #for sid in sids:
    dict1 = {"drugs": drugs, "sids": len(sids)}#dict(zip(drug,sids)
    print("The number of SIDS associated to ", drug, "is ", len(sids))
    print(dict1)
#for key, ele in zip(, test_dict.items()):
    #res[key] = dict([ele])

For Drug:  quinidine
The number of SIDS associated to  quinidine is  625
{'drugs': {'quinidine', 'memantine', 'donepezil'}, 'sids': 625}
For Drug:  memantine
The number of SIDS associated to  memantine is  181
{'drugs': {'quinidine', 'memantine', 'donepezil'}, 'sids': 181}
For Drug:  donepezil
The number of SIDS associated to  donepezil is  210
{'drugs': {'quinidine', 'memantine', 'donepezil'}, 'sids': 210}


In [167]:
def associated_studies_l2g_query(gene_id, verbose = False):
    """ example associated_studies_l2g_query(ENSG00000102908)
    """
    #construct query string and declare variables that will be sent in query
    api_query = """
    query GenePageQuery($geneId: String!) {
      geneInfo(geneId: $geneId) {
        id
        symbol
      }
      studiesAndLeadVariantsForGeneByL2G(geneId: $geneId) {
        pval
        yProbaModel
        study {
          studyId
          traitReported
          pubAuthor
          pubDate
          pmid
          nInitial
          nReplication
          hasSumstats
        }
        variant {
          rsId
          id
        }
        odds{
          oddsCI
          oddsCILower
          oddsCIUpper
        }
        beta{
          betaCI
          betaCILower
          betaCIUpper
          direction
        }
      }
    }
    """

    #set base_url for Open Targets Genetics Portal API
    base_url = "https://api.genetics.opentargets.org/graphql [api.genetics.opentargets.org]" # "http://genetics-api.opentargets.io/graphql"

    #set variables object
    variables = {"geneId": gene_id}

    #perform API call using query string and variables object
    r = requests.post(base_url, json={"query": api_query, "variables": variables})

    #check status code of GraphQL API response and print error message if code == 400
    if str(r.status_code) == "400":
        print(f"{gene_id} query status code: {r.status_code}")
    else:
        pass

    #transform API response into JSON
    api_response_as_json = json.loads(r.text)
    
    #print first element of JSON response data
    if verbose:
        print(api_response_as_json["data"]["studiesAndLeadVariantsForGeneByL2G"][0])
    #return entire JSON response data
    #return api_response_as_json
    
    # return pandas df with data
    for i in range(0,len(api_response_as_json["data"]["studiesAndLeadVariantsForGeneByL2G"])):
        df = pd.json_normalize(api_response_as_json["data"]["studiesAndLeadVariantsForGeneByL2G"][i])
        if i == 0 :
            res = df
        else:
            res = res.append(df)
    return res
